In [ ]:
""" MonkeyPie
    
    FIND FUZZY DUPLICATES FOR SONGS ACROSS ANY CODEC
    - include check better quality, including probable glitch/skip analysis
    
    - matching inc. levenshtein fuzzy-compare of the "artist + songtitle"
        and of audio profiles
        
"""

In [73]:
import win32com.client
# import sys
# import time
# import Levenshtein
# import codecs

In [77]:
def get_songsDB(
    dispatch = 'SongsDB.SDBApplication',
    shutdown = False
    ):
        SDB = win32com.client.Dispatch( dispatch )
        SDB.ShutdownAfterDisconnect = shutdown
        return SDB

def getResults_pathLike ( W32_SongsDB, matchStr ):
    results = []
    seltracks = W32_SongsDB.Database.QuerySongs( 
        f"Songs.SongPath LIKE '%{ matchStr }'")
        
    while not seltracks.EOF:
        resultItem = seltracks.Item
        results.append(resultItem)
        seltracks.Next()
        
    print( f"QUERY [ {matchStr} ] RETURNED [ { len(results)} ] RESULTS " )
    W32_SongsDB = None
    return results

In [135]:
def main():
    
    SongsDB = get_songsDB()
    dbItems_flac = getResults_pathLike ( SongsDB, ".flac" )
    
    return dbItems_flac

# if __name__ == '__main__':
flacs = main()

# takes ~8 sec to return ~8000 results from ~44000 files

QUERY [ .flac ] RETURNED [ 8149 ] RESULTS 


In [94]:
# # Get track object attributes
# dirStrs = sorted( [ attrStr for attrStr in flacs[0].__dir__() ] )
# for dirStr in dirStrs: print( dirStr )

In [110]:
attribsSelect = [
    'ID',
    'Title',
    'ArtistName',
    'SongLengthString',
    'AlbumName',
    'AlbumArtistName',
    'MusicComposer',
    'Encoder',
    'Bitrate',
    'SampleRate',
    'Channels',
    'FileLength',
    'Date',
    'LastPlayed',
    'DateAdded',
    'DateDBModified',
    'FileModified',
    'DiscNumber',
    'TrackOrder',
    'Genre',
    'Path',
    'PlayCounter',
    'SkipCount',
    'SongID',
    'SongLength',
]

In [111]:
def getAttrValPair( obj, attString, toConsole=False ):
    
    value = eval( f"obj.{attString}" )
    
    if toConsole:
        valStr = str(value)[:60]
        if len(valStr) > 59: valStr += "..."
        padd = max([ len(attrStr) for attrStr in attribsSelect ]) + 2
        print( f"{attString}:{ ' ' * (padd-len(attString)) }{valStr}" )
        
    else: return ( attString, value )

In [133]:
def fileLengthConvert( byteSize, mb = True ):
    # disagrees (less than by ~1mb) win filesys "size" field
    outSize = byteSize / 1024
    if mb: return outSize / 1024
    return outSize
    
size = fileLengthConvert( flacs[0].FileLength )
size

Decimal('34.6260051727294921875')

In [112]:
# output selected track attributes
for attrStr in attribsSelect: getAttrValPair( flacs[0], attrStr, toConsole=True )

ID:                33508
Title:             Gymnopedie for piano No. 1
ArtistName:        Tamara-Anna Cislowski
SongLengthString:  3:25
AlbumName:         The Persian Hours
AlbumArtistName:   Tamara-Anna Cislowski
MusicComposer:     Erik Satie
Encoder:           FLAC 1.3.3
Bitrate:           1416344
SampleRate:        44100
Channels:          2
FileLength:        36307998
Date:              2008-01-01 00:00:00+00:00
LastPlayed:        1899-12-30 00:00:00+00:00
DateAdded:         2021-08-27 11:33:08.384000+00:00
DateDBModified:    2021-08-27 13:09:57.089000+00:00
FileModified:      2021-08-12 01:01:45.704000+00:00
DiscNumber:        1
TrackOrder:        1
Genre:             Classical
Path:              G:\E MUSIC\flac\Tamara-Anna Cislowski\The Persian Hours\000 ...
PlayCounter:       0
SkipCount:         0
SongID:            33508
SongLength:        205000


In [134]:
# add to now playing
SDB = win32com.client.Dispatch('SongsDB.SDBApplication')
SDB.ShutdownAfterDisconnect = False
SDB.Player.PlaylistAddTrack( flacs[0] )
SDB = None